In [6]:
import scrapy
import re

BASE_URL = "http://en.wikipedia.org"

In [7]:
class NWinnerItemBio(scrapy.Item):
    link = scrapy.Field()
    name = scrapy.Field()
    mini_bio = scrapy.Field()
    image_urls = scrapy.Field()
    bio_image = scrapy.Field()
    images = scrapy.Field()

In [9]:
class NWinnerSpiderBio(scrapy.Spider):
    name = 'nwinners_minibio'
    allowed_domains = ['en.wikipedia.org']
    start_urls = ["https://en.wikipedia.org/wiki/List_of_Nobel_laureates_by_country"]
    
    def parse(self, response):
        filename = response.url.split('/')[-1]
        h2s = response.xpath('//h2')
        
        for h2 in h2s:
            country = h2.xpath('span[@class="mw-headline"]text()').extract()
            if country:
                winners = h2.xpath('following-siblng::ol[1]')
                for w in winners.xpath('li'):
                    wdata = {}
                    wdata['link'] = BASE_URL + w.xpath('a/@href').extract()[0]
                    # get_mini_bio メソッドで受賞者の人物情報ページを処理する
                    request = scrapy.Request(wdata['link'], callback=self.get_mini_bio)
                    request.meta['item'] = NWinnerItem(**wdata)
                    yield request
                            
    def get_mini_bio(self, response):
        """ 受賞者の人物情報テキストと写真を取得する """
        
        BASE_URL_ESCAPED = 'http:\/\/en.wikipedia.org'
        item = response.meta['item']
        item['image_urls'] = []
        img_src = responsr.xpath('//table[contains(@class,"inforbox")]//img/@src')
        if img_src:
            item['image_urls'] = ['http' + img_src[0].extract()]
        mini_nib = ''
        paras = response.xpath('//*[@id="mw-content-text"]/p[text() or normalize-space(.)=""]').extract()
        
        for p in paras:
            if p == '<p></p>':
                break
            mini_bio += p
        
        # ウィキリンクの補正
        mini_bio = mini_bio.replace('href="/wiki', 'href="' + BASE_URL + '/wiki')
        mini_bio = mini_bio.replace('href="#', item['link'] + '#')
        item['mini_bio'] = mini_bio
        yield item

In [11]:
from scrapy.pipelines.images import ImagesPipeline
from scrapy.exceptions import DropItem

In [13]:
class NobelImagesPipeline(ImagesPipeline):
    
    def get_media_requests(self, item, info):
        
        for image_url in item['images_urls']:
            yield scrapy.Request(image_url)
    
    def item_completed(self, results, item, info):
        
        image_paths = [x['path'] for ok, x in results if ok]
        if image_paths:
            item['bio_image'] = image_paths[0]
            
        return item